# Genomics Algorithmics project

## Development of a mapping solution over a reference genome for sequencing datas  

### Nicolas Parisot & Sergio Peignier

## Reading sequencing datas with *Biopython* package

### Genome and inversed genome of *Plasmodium falciparum*

With *Biopython*, each of the 15 *Plasmodium falciparum* chromosomes is stored in a python list under the form of a string. The python list is named **chromosomes**. Mapping operations will be realised on these chromosomes.

In [2]:
from Bio import SeqIO
from tqdm import tqdm
from Chromosome import Chromosome
from dc3 import dc3
import bwt
import tools
import mapping
import importlib

In [3]:
chromosomes:list[Chromosome] = []
chromosomes_inv:list[Chromosome] = []

with tqdm(total=30, desc="Chromosomes importation") as pbar:
    for i,record in enumerate(SeqIO.parse("SEQUENCES/P_fal_genome.fna", format="fasta")):
        # Normal chromosomes
        name = f"P_fal_chromosome_{i+1}"
        chromo = Chromosome(name,record.seq,record.id)
        chromosomes.append(chromo)
        pbar.update(1)
        
        # Inversed chromosomes
        name_inversed_comp = f"P_fal_chromosome_-{i+1}"
        inversed_comp_seq = tools.inverse_sequence(record.seq)
        chromo_inv = Chromosome(name_inversed_comp,inversed_comp_seq,record.id)
        chromosomes_inv.append(chromo_inv)
        pbar.update(1)

Chromosomes importation: 100%|██████████| 30/30 [00:12<00:00,  2.44it/s]


Number and lengths of the chromosomes are given here:

In [4]:
print(f"Chromosomes number: {len(chromosomes)}")
print("Chromosomes length, from 1 to 15:")
for i, chromo in enumerate(chromosomes):
    i += 1
    print(f"Length of the chromosome {i:>2} : {len(chromo.DNA_dol):<7} pairs of bases")
for i, chromo in enumerate(chromosomes_inv):
    i += 1
    print(f"Length of the chromosome -{i:>2} : {len(chromo.DNA_dol):<7} pairs of bases")

Chromosomes number: 15
Chromosomes length, from 1 to 15:
Length of the chromosome  1 : 640852  pairs of bases
Length of the chromosome  2 : 947103  pairs of bases
Length of the chromosome  3 : 1067972 pairs of bases
Length of the chromosome  4 : 1200491 pairs of bases
Length of the chromosome  5 : 1343558 pairs of bases
Length of the chromosome  6 : 1418243 pairs of bases
Length of the chromosome  7 : 1445208 pairs of bases
Length of the chromosome  8 : 1472806 pairs of bases
Length of the chromosome  9 : 1541736 pairs of bases
Length of the chromosome 10 : 1687657 pairs of bases
Length of the chromosome 11 : 2038341 pairs of bases
Length of the chromosome 12 : 2271495 pairs of bases
Length of the chromosome 13 : 2925237 pairs of bases
Length of the chromosome 14 : 3291937 pairs of bases
Length of the chromosome 15 : 34251   pairs of bases
Length of the chromosome - 1 : 640852  pairs of bases
Length of the chromosome - 2 : 947103  pairs of bases
Length of the chromosome - 3 : 1067972 p

The objectives of this project is to perform a **mapping** of short DNA sequences (1500000 of 100 nucleotides long) over all the chromosomes we just imported. In other words, we need to find their localisation.

Due to the length of the genome and the number of read to map, this problem can't be done with "naive" methods. Instead, we use a **string-search** algorithm, based on the Burrows Wheeler Transform (BWT) (Sources: https://www.molgen.mpg.de/3708260/bwt_fm.pdf). The read are mapped over the BWT of all the chromosomes. To that extend, we need to compute the BWT of very large strings which can be very time consuming. The **DC3 algorithm** allows us to compute very efficiently suffix array of strings and the we use it to compute the BWT.

_____________________

## Suffix array of the genome with DC3 algorithm
The following lines of code are used to import the suffix table computed from the DC3 algorithm. Suffix tables are stored in npy files, one of the best way of storage for list <p>
Source: https://stackoverflow.com/questions/9619199/-way-to-preserve-numpy-arrays-on-disk

In [5]:
for chromo in chromosomes+chromosomes_inv:
    print(chromo.file_name)
    if chromo.suffix_table is None:
        dc3result = dc3(chromo.DNA_dol)
        chromo.export_dc3_result(dc3result)

P_fal_chromosome_1
P_fal_chromosome_2
P_fal_chromosome_3
P_fal_chromosome_4
P_fal_chromosome_5
P_fal_chromosome_6
P_fal_chromosome_7
P_fal_chromosome_8
P_fal_chromosome_9
P_fal_chromosome_10
P_fal_chromosome_11
P_fal_chromosome_12
P_fal_chromosome_13
P_fal_chromosome_14
P_fal_chromosome_15
P_fal_chromosome_-1
P_fal_chromosome_-2
P_fal_chromosome_-3
P_fal_chromosome_-4
P_fal_chromosome_-5
P_fal_chromosome_-6
P_fal_chromosome_-7
P_fal_chromosome_-8
P_fal_chromosome_-9
P_fal_chromosome_-10
P_fal_chromosome_-11
P_fal_chromosome_-12
P_fal_chromosome_-13
P_fal_chromosome_-14
P_fal_chromosome_-15


## Burrows Wheeler Transform (BWT) of the genome
This chunk creates (or imports) the BWT transform of every chromosomes.

In [6]:
for chromo in chromosomes+chromosomes_inv:
    print(chromo.file_name)
    if chromo.bwt is None:
        bwt_result = bwt.bwt(str(chromo.DNA_dol),chromo.suffix_table)
        bwt_result = bwt_result.replace("$", "")
        chromo.export_bwt_result(bwt_result)


P_fal_chromosome_1
P_fal_chromosome_2
P_fal_chromosome_3
P_fal_chromosome_4
P_fal_chromosome_5
P_fal_chromosome_6
P_fal_chromosome_7
P_fal_chromosome_8
P_fal_chromosome_9
P_fal_chromosome_10
P_fal_chromosome_11
P_fal_chromosome_12
P_fal_chromosome_13
P_fal_chromosome_14
P_fal_chromosome_15
P_fal_chromosome_-1
P_fal_chromosome_-2
P_fal_chromosome_-3
P_fal_chromosome_-4
P_fal_chromosome_-5
P_fal_chromosome_-6
P_fal_chromosome_-7
P_fal_chromosome_-8
P_fal_chromosome_-9
P_fal_chromosome_-10
P_fal_chromosome_-11
P_fal_chromosome_-12
P_fal_chromosome_-13
P_fal_chromosome_-14
P_fal_chromosome_-15


Next we compute or import the rank matrix of each chromosome. Rank matrices are large python dictionnaries storing for a chromosome, the rank array of each of its four nucleotides "A", "C", "G", "T" and for the end of string character "$".<p>
Rank matrices are very useful for our *string_search* function in the mapping module. They allow us to localise substrings with $O(1)$ complexity (again the source: https://www.molgen.mpg.de/3708260/bwt_fm.pdf).

In [7]:
for chromo in chromosomes+chromosomes_inv:
    print(chromo.file_name)
    if chromo.rank_mat is None:
        print(len(chromo.bwt))
        rank_mat = bwt.create_rank_mat(chromo.bwt)
        print("Done !")
        chromo.export_rank_matrix_result(rank_mat) 
        print("Exported !")

P_fal_chromosome_1
P_fal_chromosome_2
P_fal_chromosome_3
P_fal_chromosome_4
P_fal_chromosome_5
P_fal_chromosome_6
P_fal_chromosome_7
P_fal_chromosome_8
P_fal_chromosome_9
P_fal_chromosome_10
P_fal_chromosome_11
P_fal_chromosome_12
P_fal_chromosome_13
P_fal_chromosome_14
P_fal_chromosome_15
P_fal_chromosome_-1
P_fal_chromosome_-2
P_fal_chromosome_-3
P_fal_chromosome_-4
P_fal_chromosome_-5
P_fal_chromosome_-6
P_fal_chromosome_-7
P_fal_chromosome_-8
P_fal_chromosome_-9
P_fal_chromosome_-10
P_fal_chromosome_-11
P_fal_chromosome_-12
P_fal_chromosome_-13
P_fal_chromosome_-14
P_fal_chromosome_-15


## Reads importation
*Reads* sequence and identifier are extracted with *Biopython* and stored in a python list **reads**. The sequencing produced 100000 *reads* for each of the 15 chromomoses for a total of 1500000 *reads*. All the *reads* have the same length: 100 pairs of bases. 

In [20]:
reads = []
with tqdm(total=1500000, desc="Reads importation") as pbar:
    for record in SeqIO.parse("SEQUENCES/P_fal_reads.fq", format="fastq"):
        reads.append((str(record.seq), record.id))
        pbar.update(1)

Reads importation: 100%|██████████| 1500000/1500000 [00:17<00:00, 84710.49it/s]


In [21]:
print(f"Number of reads: {len(reads)}")
print(f"Reads length: {len(reads[0][0])}")

Number of reads: 1500000
Reads length: 100
<class 'str'>


In [17]:
importlib.reload(mapping)
def test():
    with tqdm(total=5000, desc="Reads importation") as pbar:
        for i in range(5000):
            read = str(reads[i][0])
            k_reads = mapping.cut_read_to_kmer(read,20)
            k_reads_locs = [mapping.string_search(k_read, chromosomes[0]) for k_read in k_reads]
            loc_read = mapping.link_kmer(k_reads, k_reads_locs)
            if not loc_read[1] == -1:
                return loc_read
            pbar.update(1)
            
print(test())
# import cProfile
# cProfile.run("test()")

Reads importation:   0%|          | 8/5000 [00:00<00:37, 134.10it/s]

[178936 179004]


## Mapping algorithm
To find the location(s) of a given read on a genome, we start with the cutting of a read into smaller pieces, called **kmers**.
Then we find all the locations of these kmers and we "reconstruct" the read to find it's most precise localisation. <p>
The length of the kmers is set to 20 which seems to be a good trade-off between precision and mapping speed (the smaller a kmer is, the better the mapping can be because it is less sensitive to mutations).<p>
The localisations having the minimal number of mismatched kmer is considered as the most probable localisations and therefor are the one conserved in the results.<p>

In [ ]:
for read_info in tqdm(reads):
    read_seq = read_info[0]
    read_id = read_info[0]
    k_read = mapping.cut_read_to_kmer(read_seq, 20)
    k_locs = []
    for chromo in chromosomes:
        pass 
        # TODO remplir k_locs avec les resultats de la string search +
        # une clé permettant de retrouver le chromosome de ces positions
    # TODO Pour chaque array dans klocs, reconstruire les reads et écrire
    # le read, ses positions et le chromosome de ces positions dans un fichier 
        